In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
### this is a comment 

In [41]:
accre_data = pd.read_csv(r'C:\Users\chris\data_science_projects\data-question-2-dream-crushers\july-2017-thru-june-2018.txt', delimiter='|')
accre_data.head(n=20)

C:\Users\chris\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21
5,15811618,15811618,accre,production,cms_stage2,cms_stage1,59298,autocms,9209.0,2017-07-01T05:00:01,...,COMPLETED,1,1,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1063,skim_test
6,15811619,15811619,accre,production,cms_stage2,cms,31000,tuos,126986.0,2017-07-01T05:00:02,...,COMPLETED,1,1,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1062,skim_test
7,15811621,15811621,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:22,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_33ee450ebdc4
8,15811624,15811624,accre,production,cms_stage2,cms_stage1,59298,autocms,9209.0,2017-07-01T05:01:01,...,COMPLETED,1,1,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp670,skim_test
9,15811625,15811625,accre,production,cms_stage2,cms,31000,tuos,126986.0,2017-07-01T05:01:02,...,COMPLETED,1,1,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1078,skim_test


In [42]:
accre_data['State'].value_counts()

COMPLETED              7150200
CANCELLED by 29607     3354734
FAILED                  902907
CANCELLED by 29605      355327
TIMEOUT                 160402
CANCELLED by 112888     105801
CANCELLED by 233674      66809
CANCELLED by 207302      44886
CANCELLED by 0           29569
CANCELLED by 551122      24607
CANCELLED by 53988       19998
CANCELLED by 233240      17531
CANCELLED by 237602      17013
CANCELLED by 342713      16950
CANCELLED by 501421      16363
CANCELLED by 29789       14925
PREEMPTED                13425
CANCELLED by 144605      12054
CANCELLED by 169859      11882
CANCELLED by 342288      10227
CANCELLED by 497968       9284
CANCELLED by 200524       7099
CANCELLED by 171302       6349
CANCELLED by 454080       5956
CANCELLED by 115292       5643
CANCELLED by 154990       5162
CANCELLED by 664085       5073
CANCELLED by 59223        4985
CANCELLED by 229206       4860
CANCELLED by 237813       4624
                        ...   
CANCELLED by 645144          2
CANCELLE

In [91]:
Job_State=accre_data['State']

In [92]:
print(Job_State)

0           COMPLETED
1           COMPLETED
2           COMPLETED
3           COMPLETED
4           COMPLETED
5           COMPLETED
6           COMPLETED
7           COMPLETED
8           COMPLETED
9           COMPLETED
10          COMPLETED
11          COMPLETED
12          COMPLETED
13          COMPLETED
14          COMPLETED
15          COMPLETED
16            TIMEOUT
17          COMPLETED
18          COMPLETED
19          COMPLETED
20          COMPLETED
21          COMPLETED
22          COMPLETED
23          COMPLETED
24          COMPLETED
25          COMPLETED
26            TIMEOUT
27          COMPLETED
28          COMPLETED
29          COMPLETED
              ...    
12499359       FAILED
12499360    COMPLETED
12499361       FAILED
12499362       FAILED
12499363    COMPLETED
12499364    COMPLETED
12499365    COMPLETED
12499366    COMPLETED
12499367    COMPLETED
12499368    COMPLETED
12499369    COMPLETED
12499370    COMPLETED
12499371    COMPLETED
12499372    COMPLETED
12499373  

# Calculating the proportion of failed jobs to total jobs

In [99]:
total_failures = 902907

In [103]:
print(total_failures / len(Job_State))

0.0722360909001232


# About 7% of all jobs fail (902,907 out of 12,499,389 total jobs failed).

# Central Question: Are these 7% of failures coming from a disproportionate number of particular nodes?

# Before we answer that question, we will only consider jobs that used 1 node (which 98.97% of all jobs).

In [107]:
accre_data['NNodes'].value_counts()

1      12370952
0        108668
2          9344
3          2746
4          2357
5           811
100         741
6           497
8           488
36          273
50          237
7           202
10          157
82          144
16          131
25          122
64          112
12          102
32           82
9            73
20           62
13           61
11           60
15           56
14           52
17           45
19           45
18           42
30           37
128          32
         ...   
96            2
61            2
76            2
98            2
83            2
77            1
89            1
188           1
136           1
132           1
131           1
125           1
124           1
118           1
117           1
116           1
115           1
114           1
111           1
109           1
84            1
85            1
104           1
86            1
97            1
200           1
95            1
88            1
93            1
94            1
Name: NNodes, Length: 12

In [113]:
One_Node= accre_data[accre_data['NNodes'] == 1]

In [114]:
print(One_Node)

             JobID  JobIDRaw Cluster   Partition      Account        Group  \
0         15806649  15806649   accre  production      h_vuiis      h_vuiis   
1         15806664  15806664   accre  production      h_vuiis      h_vuiis   
2         15806670  15806670   accre  production      h_vuiis      h_vuiis   
3         15806671  15806671   accre  production      h_vuiis      h_vuiis   
4         15811617  15811617   accre  production  cms_samtest  cms_samtest   
5         15811618  15811618   accre  production   cms_stage2   cms_stage1   
6         15811619  15811619   accre  production   cms_stage2          cms   
7         15811621  15811621   accre  production  cms_samtest  cms_samtest   
8         15811624  15811624   accre  production   cms_stage2   cms_stage1   
9         15811625  15811625   accre  production   cms_stage2          cms   
10        15811628  15811628   accre  production      h_vuiis      h_vuiis   
11        15811629  15811629   accre  production      h_vuiis   

In [115]:
accre_data['NodeList'].value_counts()

None assigned                                                                                                                                                                                                                                                                                                                                                                                                                             3470227
vmp1409                                                                                                                                                                                                                                                                                                                                                                                                                                     92419
vmp1242                                                                                                                             

In [123]:
accre_data[(accre_data['NodeList'] =="vmp1409") | (accre_data['State'] == "FAILED")]

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
386,15812080,15812080,accre,production,segrest_group,segrest_group,20482,songhd,633620.0,2017-07-01T05:23:12,...,FAILED,1,1,1,8Gn,NaN,"cpu=1,mem=8G,node=1",2-00:00:00,vmp1106,dock-t
441,15812174,15812174,accre,production,segrest_group,segrest_group,20482,songhd,633620.0,2017-07-01T05:43:04,...,FAILED,1,1,1,8Gn,NaN,"cpu=1,mem=8G,node=1",08:00:00,vmp1119,rep-A
453,15812190,15812190,accre,production,segrest_group,segrest_group,20482,songhd,633620.0,2017-07-01T05:46:34,...,FAILED,1,1,1,8Gn,NaN,"cpu=1,mem=8G,node=1",08:00:00,vmp1122,rep-A
537,15812298,15812298,accre,production,segrest_group,segrest_group,20482,songhd,633620.0,2017-07-01T06:02:31,...,FAILED,1,1,1,8Gn,NaN,"cpu=1,mem=8G,node=1",2-00:00:00,vmp1090,dock-t
543,15812304,15812304,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-01T06:03:33,...,FAILED,1,1,1,10Gn,NaN,"cpu=1,mem=10G,node=1",10:00:00,vmp1091,JEFFERSON-x-230761-x-230761-x-401-x-MaCRUISE_v...
544,15812305,15812305,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-01T06:03:33,...,FAILED,1,1,1,10Gn,NaN,"cpu=1,mem=10G,node=1",10:00:00,vmp1097,JEFFERSON-x-230831-x-230831-x-401-x-MaCRUISE_v...
545,15812306,15812306,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-01T06:03:33,...,FAILED,1,1,1,10Gn,NaN,"cpu=1,mem=10G,node=1",10:00:00,vmp1029,JEFFERSON-x-230830-x-230830-x-501-x-MaCRUISE_v...
546,15812307,15812307,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-01T06:03:34,...,FAILED,1,1,1,10Gn,NaN,"cpu=1,mem=10G,node=1",10:00:00,vmp1073,JEFFERSON-x-131175-x-131175-x-501-x-MaCRUISE_v...
547,15812308,15812308,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-01T06:03:34,...,FAILED,1,1,1,10Gn,NaN,"cpu=1,mem=10G,node=1",10:00:00,vmp1007,JEFFERSON-x-230833-x-230833-x-201-x-MaCRUISE_v...
548,15812309,15812309,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-01T06:03:34,...,FAILED,1,1,1,10Gn,NaN,"cpu=1,mem=10G,node=1",10:00:00,vmp1089,JEFFERSON-x-230828-x-230828-x-401-x-MaCRUISE_v...


# Let's look @ at the failure rate of the most used node--vmp1409--before we look at all nodes. 

In [128]:
accre_data[(accre_data['State'] == "FAILED") & (accre_data["NodeList"] == "vmp1409")]

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
3151714,18971910_0,18971911,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151715,18971910_1,18971912,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151716,18971910_2,18971913,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151717,18971910_3,18971914,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151718,18971910_4,18971915,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151719,18971910_5,18971916,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151720,18971910_6,18971917,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151721,18971910_7,18971918,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151722,18971910_8,18971919,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm
3151723,18971910_9,18971920,accre,production,accre,accre,36014,frenchwr,112888.0,2017-09-25T20:46:25,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",00:15:00,vmp1409,singularity-job.slurm


# Node vmp1409 was used to execute the most jobs: 3,470,227 jobs. It had 1,119 failures. The frequency of failure was less than .03%--far below the average fail rate. Because this node executed a sizeable percentage of all jobs and had such a low failure rate, I predict that there is a node with a disproportionately high failure rate that is skewing the overall average.

# Let's keep analyzing each node.

In [130]:
accre_data[(accre_data['State'] == "FAILED") & (accre_data["NodeList"] == "vmp1242")]

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
40474,15857888,15857888,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-04T22:32:29,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_2b3809013209
80967,15898381,15898381,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T19:25:13,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_dc48d21e6d4c
80968,15898382,15898382,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T19:25:13,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_dc48d21e6d4c
82287,15899701,15899701,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T21:32:54,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_58691b50faff
82288,15899702,15899702,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T21:32:55,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_6fd8ced5f60f
82292,15899706,15899706,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T21:32:55,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_58691b50faff
82293,15899707,15899707,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T21:32:55,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_fdf011a037bc
82294,15899708,15899708,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T21:32:55,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_58691b50faff
82296,15899710,15899710,accre,production,cms_samtest,cms,31000,uscms010,NaN,2017-07-06T21:32:55,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_6fd8ced5f60f
100900,15919242,15919242,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-07T20:41:42,...,FAILED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_a20b6513bad9


# Node 1242 has a failure rate of 5.64%.

In [131]:
accre_data[(accre_data['State'] == "FAILED") & (accre_data["NodeList"] == "vmp1257")]

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
21344,15838734_3,15838737,accre,production,h_biostat_student,h_biostat_student,59331,schluedj,169859.0,2017-07-03T15:06:30,...,FAILED,1,4,1,70Gn,NaN,"cpu=1,mem=70G,node=1",12:00:00,vmp1257,Barycenter_Sim.slurm
21372,15838762_9,15838771,accre,production,h_biostat_student,h_biostat_student,59331,schluedj,169859.0,2017-07-03T15:10:28,...,FAILED,1,4,1,70Gn,NaN,"cpu=1,mem=70G,node=1",12:00:00,vmp1257,Barycenter_Sim.slurm
21399,15838800_5,15838805,accre,production,h_biostat_student,h_biostat_student,59331,schluedj,169859.0,2017-07-03T15:15:41,...,FAILED,1,4,1,70Gn,NaN,"cpu=1,mem=70G,node=1",12:00:00,vmp1257,Barycenter_Sim.slurm
21428,15838839_2,15838842,accre,production,h_biostat_student,h_biostat_student,59331,schluedj,169859.0,2017-07-03T15:19:02,...,FAILED,1,4,1,70Gn,NaN,"cpu=1,mem=70G,node=1",12:00:00,vmp1257,Barycenter_Sim.slurm
21473,15838882_5,15838887,accre,production,h_biostat_student,h_biostat_student,59331,schluedj,169859.0,2017-07-03T15:28:44,...,FAILED,1,4,1,70Gn,NaN,"cpu=1,mem=70G,node=1",12:00:00,vmp1257,Barycenter_Sim.slurm
21538,15838947_5,15838952,accre,production,h_biostat_student,h_biostat_student,59331,schluedj,169859.0,2017-07-03T15:41:29,...,FAILED,1,4,1,70Gn,NaN,"cpu=1,mem=70G,node=1",12:00:00,vmp1257,Barycenter_Sim.slurm
28425,15845816_9,15845839,accre,production,p_csb_meiler,p_csb_meiler,59420,sauermf,85636.0,2017-07-03T21:29:37,...,FAILED,1,2,2,16Gc,NaN,"cpu=2,mem=32G,node=1",4-04:00:00,vmp1257,rsvtri_recon_mpi
28426,15845816_10,15845840,accre,production,p_csb_meiler,p_csb_meiler,59420,sauermf,85636.0,2017-07-03T21:29:37,...,FAILED,1,2,2,16Gc,NaN,"cpu=2,mem=32G,node=1",4-04:00:00,vmp1257,rsvtri_recon_mpi
28427,15845816_11,15845841,accre,production,p_csb_meiler,p_csb_meiler,59420,sauermf,85636.0,2017-07-03T21:29:37,...,FAILED,1,2,2,16Gc,NaN,"cpu=2,mem=32G,node=1",4-04:00:00,vmp1257,rsvtri_recon_mpi
28572,15845986,15845986,accre,production,capra_lab_csb,capra_lab,20429,sivleyrm,156369.0,2017-07-03T21:43:28,...,FAILED,1,1,1,5Gn,NaN,"cpu=1,mem=5G,node=1",4-00:00:00,vmp1257,PSB_Structural_Pipeline_ENSP00000366297.4_1_A_...


# Node1257 has a failure rate of 1.66%.

In [133]:
accre_data[(accre_data['State'] == "FAILED") & (accre_data["NodeList"] == "vmp1001")]

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
22735,15840149,15840149,accre,production,h_taylor,h_taylor,59339,boydb1,177470.0,2017-07-03T17:21:21,...,FAILED,1,1,1,8Gn,NaN,"cpu=1,mem=8G,node=1",02:00:00,vmp1001,NewhouseMDDHx-x-2556-x-2556b-x-RSFC_CONN_v0.slurm
26631,15844045,15844045,accre,production,h_biostat_student,h_biostat_student,59331,fletchs2,144605.0,2017-07-03T20:01:29,...,FAILED,1,1,1,4Gn,NaN,"cpu=1,mem=4G,node=1",01:00:00,vmp1001,MCAR_2
26650,15844064,15844064,accre,production,h_biostat_student,h_biostat_student,59331,fletchs2,144605.0,2017-07-03T20:01:31,...,FAILED,1,1,1,4Gn,NaN,"cpu=1,mem=4G,node=1",01:00:00,vmp1001,MCAR_21
26669,15844083,15844083,accre,production,h_biostat_student,h_biostat_student,59331,fletchs2,144605.0,2017-07-03T20:01:32,...,FAILED,1,1,1,4Gn,NaN,"cpu=1,mem=4G,node=1",01:00:00,vmp1001,MCAR_40
26688,15844102,15844102,accre,production,h_biostat_student,h_biostat_student,59331,fletchs2,144605.0,2017-07-03T20:01:33,...,FAILED,1,1,1,4Gn,NaN,"cpu=1,mem=4G,node=1",01:00:00,vmp1001,MCAR_59
27021,15844435,15844435,accre,production,h_biostat_student,h_biostat_student,59331,fletchs2,144605.0,2017-07-03T20:17:51,...,FAILED,1,1,1,4Gn,NaN,"cpu=1,mem=4G,node=1",01:00:00,vmp1001,MNAR_2
27421,15844835,15844835,accre,production,h_biostat_student,h_biostat_student,59331,fletchs2,144605.0,2017-07-03T21:03:13,...,FAILED,1,1,1,4Gn,NaN,"cpu=1,mem=4G,node=1",01:00:00,vmp1001,MCAR_6
85401,15903743,15903743,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-07T02:00:07,...,FAILED,1,1,1,4Gn,NaN,"cpu=1,mem=4G,node=1",1-00:00:00,vmp1001,ZALD_TTS-x-127117-x-127117-x-201-x-Temporal_Lo...
88843,15907185,15907185,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-07T06:03:29,...,FAILED,1,1,1,10Gn,NaN,"cpu=1,mem=10G,node=1",10:00:00,vmp1001,JEFFERSON-x-230321-x-230321-x-401-x-MaCRUISE_v...
110662,15929004,15929004,accre,production,cms_stage2,cms,31000,tuos,126986.0,2017-07-08T05:08:20,...,FAILED,1,1,1,2Gc,NaN,"cpu=1,mem=2G,node=1",12:00:00,vmp1001,skim_test


In [ ]:
# Commentary

In [132]:
accre_data[(accre_data['State'] == "FAILED") & (accre_data["NodeList"] == "vmp1298")]

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
37430,15854844,15854844,accre,production,rokaslab,rokaslab,36087,lindal,172079.0,2017-07-04T17:13:13,...,FAILED,1,8,8,10Gc,NaN,"cpu=8,mem=80G,node=1",23:00:00,vmp1298,aln_mm.sbatch
37623,15855037,15855037,accre,production,rokaslab,rokaslab,36087,lindal,172079.0,2017-07-04T17:38:54,...,FAILED,1,8,8,10Gc,NaN,"cpu=8,mem=80G,node=1",23:00:00,vmp1298,aln_mm.sbatch
231147,16049489,16049489,accre,production,biostat_it,biostat_it,59248,liuz,40091.0,2017-07-11T21:37:23,...,FAILED,1,10,1,200Gn,NaN,"cpu=1,mem=200G,node=1",23:00:00,vmp1298,bioView2.slurm
231200,16049542,16049542,accre,production,biostat_it,biostat_it,59248,liuz,40091.0,2017-07-11T21:37:42,...,FAILED,1,10,1,200Gn,NaN,"cpu=1,mem=200G,node=1",23:00:00,vmp1298,bioView11.slurm
231726,16050068,16050068,accre,production,sbcs,sbcs,59291,hej4,121574.0,2017-07-11T21:55:33,...,FAILED,1,1,1,20Gn,NaN,"cpu=1,mem=20G,node=1",10:00:00,vmp1298,cov_22.slum
372114,16185863_38,16190456,accre,production,p_csb_meiler,p_csb_meiler,59420,sauermf,85636.0,2017-07-13T18:08:34,...,FAILED,1,2,2,12Gc,NaN,"cpu=2,mem=24G,node=1",20:00:00,vmp1298,1a29_1cff_recon_mpi
586793,16405136,16405136,accre,production,cms,cms,31000,cmslocal,9202.0,2017-07-17T20:40:45,...,FAILED,1,2,1,28000Mn,NaN,"cpu=1,mem=28000M,node=1",2-00:00:00,vmp1298,bl_069c7c5b4684
586815,16405158,16405158,accre,production,cms,cms,31000,cmslocal,9202.0,2017-07-17T20:41:06,...,FAILED,1,2,1,28000Mn,NaN,"cpu=1,mem=28000M,node=1",2-00:00:00,vmp1298,bl_091e4c0a59b3
647517,16465861,16465861,accre,production,h_vuiis,h_vuiis,36052,vuiiscci,NaN,2017-07-19T01:05:13,...,FAILED,1,1,1,7Gn,NaN,"cpu=1,mem=7G,node=1",04:00:00,vmp1298,IXI-x-IXI228-Guys-0822-x-IXI228-Guys-0822-1-x-...
647779,16466123,16466123,accre,production,h_vuiis,h_vuiis,36052,vuiiscci,NaN,2017-07-19T01:08:52,...,FAILED,1,1,1,9000Mn,NaN,"cpu=1,mem=9000M,node=1",12:00:00,vmp1298,BLSA-x-BLSA_5950-x-BLSA_5950_04-0_10-x-REST-x-...
